In [ ]:
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime, timedelta
from garminconnect import Garmin

In [ ]:
LOOKBACK_WINDOW = 3
START_DATE = datetime.today().date()

In [ ]:
email = os.getenv('garmin_email')
password = os.environ.get('garmin_pwd')

In [ ]:
api = Garmin(email, password)
api.login()

In [ ]:
# steps data
steps_vals = {
    'startGMT': [],
    'endGMT': [],
    'steps': [],
    'primaryActivityLevel': [],
    'activityLevelConstant': []
}
for i in range(LOOKBACK_WINDOW):
    steps_l = api.get_steps_data((START_DATE - timedelta(days=i)).isoformat())
    for entry in steps_l:
        for key in entry:
            steps_vals[key].append(entry[key])
# steps df has info on the number of steps, activity level in 15 min interval
steps_df = pd.DataFrame(steps_vals)

In [ ]:
# heart rate data
hr_vals = {
    'recordGMT': [],
    'heartRate': []
}

for i in range(LOOKBACK_WINDOW):
    heart_l = api.get_heart_rates((START_DATE - timedelta(days=i)).isoformat())['heartRateValues']
    for entry in heart_l:
        timestamp = datetime.fromtimestamp(int(str(entry[0])[:10]))
        hr_vals['recordGMT'].append(timestamp)
        hr_vals['heartRate'].append(entry[1])

# heart rate data available at a 2 minute interval
hr_df = pd.DataFrame(hr_vals)

In [ ]:
# daily resting heart data
rhr_vals = {
    'calendarDate': [],
    'restingHeartRate': []
}

for i in range(LOOKBACK_WINDOW):
    rhr_dict = api.get_rhr_day((START_DATE - timedelta(days=i)).isoformat())['allMetrics']['metricsMap']['WELLNESS_RESTING_HEART_RATE'][0]
    rhr_vals['calendarDate'].append(rhr_dict['calendarDate'])
    rhr_vals['restingHeartRate'].append(rhr_dict['value'])

# resting heart rate available daily
rhr_df = pd.DataFrame(rhr_vals)

In [ ]:
# hydration data is missing a lot of data - is it manual input?
# hydration daily

In [ ]:
# sleep data

# daily sleep data
daily_sleep_vals = {
    'calendarDate': [],
    'sleepTimeSeconds': [],
    'napTimeSeconds': [],
    'sleepStartTimestampGMT': [],
    'sleepEndTimestampGMT': [],
    'deepSleepSeconds': [],
    'lightSleepSeconds': []
}

# sleep movement data - every minute
sleep_movement_vals = {
    'startGMT': [],
    'endGMT': [],
    'activityLevel': []
}

# sleep levels data - random timeframe
sleep_level_vals = {
    'startGMT': [],
    'endGMT': [],
    'activityLevel': []
}

# sleep respiration - every two minutes
sleep_resp_vals = {
    'startTimeGMT': [],
    'respirationValue': [],
}

# sleep stress - every three minutes
sleep_stress_vals = {
    'startGMT': [],
    'value': []
}

for i in range(LOOKBACK_WINDOW):
    sleep_api = api.get_sleep_data((START_DATE - timedelta(days=i)).isoformat())
    # daily sleep
    daily_sleep_dict = sleep_api['dailySleepDTO']
    for key, value in daily_sleep_dict.items():
        if key in daily_sleep_vals.keys():
            if 'GMT' in key:
                value = datetime.fromtimestamp(int(str(value)[:10]))
            daily_sleep_vals[key].append(value)
    
    # sleep movement
    sleep_movement_l = sleep_api['sleepMovement']
    for entry in sleep_movement_l:
        for k, v in entry.items():
            sleep_movement_vals[k].append(v)
    
    # sleep activity levels
    sleep_act_l = sleep_api['sleepLevels']
    for entry in sleep_act_l:
        for k,v in entry.items():
            sleep_level_vals[k].append(v)
            
    # sleep respiration
    sleep_resp_l = sleep_api['wellnessEpochRespirationDataDTOList']
    for entry in sleep_resp_l:
        sleep_resp_vals['startTimeGMT'].append(datetime.fromtimestamp(int(str(entry['startTimeGMT'])[:10])))
        sleep_resp_vals['respirationValue'].append(entry['respirationValue'])
    
    # sleep stress
    sleep_stress_l = sleep_api['sleepStress']
    for entry in sleep_stress_l:
        sleep_stress_vals['startGMT'].append(datetime.fromtimestamp(int(str(entry['startGMT'])[:10])))
        sleep_stress_vals['value'].append(entry['value'])
            
daily_sleep_df = pd.DataFrame(daily_sleep_vals)
sleep_movement_df = pd.DataFrame(sleep_movement_vals)
sleep_activity_lvl_df = pd.DataFrame(sleep_level_vals)
sleep_resp_df = pd.DataFrame(sleep_resp_vals)
sleep_stress_df = pd.DataFrame(sleep_stress_vals)

In [ ]:
# stress data - every three minutes
# body battery - every three minutes
stress_vals = {
    'timestampGMT': [],
    'stressLevel': []
}

battery_vals = {
    'timestampGMT': [],
    'bodyBatteryStatus': [],
    'bodyBatteryLevel': [],
    'bodyBatteryVersion': []
}

for i in range(LOOKBACK_WINDOW):
    stress_api = api.get_stress_data((START_DATE - timedelta(days=i)).isoformat())
    #stress parsing
    stress_l = stress_api['stressValuesArray']
    for entry in stress_l:
        timestamp = datetime.fromtimestamp(int(str(entry[0])[:10]))
        stress_vals['timestampGMT'].append(timestamp)
        stress_vals['stressLevel'].append(entry[1])
        
    #body battery parsing
    body_battery_l = stress_api['bodyBatteryValuesArray']
    for entry in body_battery_l:
        timestamp = datetime.fromtimestamp(int(str(entry[0])[:10]))
        battery_vals['timestampGMT'].append(timestamp)
        battery_vals['bodyBatteryStatus'].append(entry[1])
        battery_vals['bodyBatteryLevel'].append(entry[2])
        battery_vals['bodyBatteryVersion'].append(entry[3])
        
stress_df = pd.DataFrame(stress_vals)
body_battery_df = pd.DataFrame(battery_vals)